In [2]:
import os

import pandas as pd

# Dataset preparation for pretrained open ai model

In [16]:

df = pd.read_csv("../dataset/input/raw/emails_set_batch.csv", index_col=0)
df = df[["text", "is_phishing"]]
df = df.rename(columns={"text": "prompt", "is_phishing": "completion"})
df = df.dropna()

df.to_json("../dataset/input/emails_set_batch.jsonl", orient='records', lines=True)

In [ ]:
!openai tools fine_tunes.prepare_data -f '../dataset/input/emails_set_batch.jsonl' -q

### Merge files made by open ai preparation tool

In [18]:
# Load preprocessed datasets and combine them together for pretrained model as it does not require any prior training

prepared_valid_jsonl = '../dataset/input/emails_set_batch_prepared_valid.jsonl'
prepared_train_jsonl = '../dataset/input/emails_set_batch_prepared_train.jsonl'

if os.path.exists(prepared_valid_jsonl):
    df_first_batch = pd.read_json(prepared_valid_jsonl, lines=True)
    df_second_batch = pd.read_json(prepared_train_jsonl, lines=True)
    df = pd.concat([df_first_batch, df_second_batch], ignore_index=True)
    df.to_json('../dataset/input/emails_set_batch.jsonl', orient='records', lines=True)

    os.remove(prepared_valid_jsonl)
    os.remove(prepared_train_jsonl)


# Main dataset preparation

In [5]:
df_mixed = pd.read_csv("../dataset/input/raw/emails_set_batch.csv", index_col=0)
df_phishing = pd.read_csv("../dataset/input/raw/phishing_emails_formatted.csv", index_col=False)
df_legitimate = pd.read_csv("../dataset/input/raw/spam_ham_emails.csv", index_col=0)

df_legitimate["is_phishing"] = 0

merged_df = pd.concat([df_mixed, df_phishing, df_legitimate], ignore_index=True)

merged_df = merged_df.loc[:, ['text', 'is_phishing']]

merged_df.to_csv('../dataset/input/emails_set.csv')

merged_df = merged_df.rename(columns={'text': 'prompt', 'is_phishing': 'completion'})

merged_df.to_json('../dataset/input/emails_set.jsonl', orient='records', lines=True)